In [1]:
import re
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

df_train = pd.read_csv('D:/_Barq/HDPSR/train.csv', encoding="ISO-8859-1")
num_train = df_train.shape[0]
df_all = pd.read_csv('D:/_Barq/HDPSR/df_all.csv')
info = df_all.product_info


In [35]:
terms = []
for idx in range(0,len(info)-1):
    terms.append(info[idx].split())
    if not idx%60000:
        print(idx)


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000


In [ ]:
# Corpus and dictionary
from gensim import corpora, models, similarities, matutils

dictionary = corpora.Dictionary(terms)
dictionary.save('D:/_Barq/HDPSR/all_info.dict') # store the dictionary, for future reference

corpus = [dictionary.doc2bow(term) for term in terms]
corpora.MmCorpus.serialize('D:/_Barq/HDPSR/all_info.mm', corpus) # store to disk, for later use
print(dictionary)


In [47]:
dictionary = corpora.Dictionary.load('D:/_Barq/HDPSR/all_info.dict')
corpus = corpora.MmCorpus('D:/_Barq/HDPSR/all_info.mm')
print('Dictionary and corpus were loaded.')


Discriptions LDA model was constructed . 
Discriptions LDA model was saved.


In [131]:
TfidfModel = models.TfidfModel(corpus)
tf_idf_corpus = TfidfModel[corpus]

lda = models.LsiModel(tf_idf_corpus, id2word=dictionary, num_topics=100)
print('LDA model was constructed . ')

doc_lda = matutils.corpus2dense(lda[corpus], 100)

lda.save('D:/_Barq/HDPSR/all_info.lda')
print('LDA model was saved.')

LDA model was constructed . 
LDA model was saved.


In [134]:

X_train = doc_lda[:,:num_train].T
X_test = doc_lda[:,num_train-1:].T
y_train = df_all.iloc[:num_train].relevance.values
id_test = df_all.iloc[num_train:].id.values

rf = RandomForestRegressor(n_estimators=15, max_depth=7, random_state=0)
clf = BaggingRegressor(rf, n_estimators=30, max_samples=0.1, random_state=25)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

results = pd.DataFrame({"id": id_test, "relevance": y_pred})
results.to_csv('D:/_Barq/HDPSR/LDA_RFR.csv',index=False)

In [133]:
doc_lda.shape

(100, 240759)